<a href="https://colab.research.google.com/github/alanbfgit/AI_May25/blob/main/wellsrColab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from IPython import get_ipython
from IPython.display import display
# %%
from google.colab import drive
#drive.mount('/content/drive')
%cd /content/drive/MyDrive
!git clone https://github.com/RunpengDai/ICSA_DLcourse.git
!pip install pypdf
!pip install chromadb
#!pip install langchain
#!pip install langchain_experimental
#!pip install langchain_community
#!pip install langgraph
##!pip install langchain_text_splitter
#!pip install langchain_core
#!pip install langchain_huggingface
#!pip install huggingface_hub
import os
import pandas as pd
from huggingface_hub import InferenceClient
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from langgraph.graph import START, END, StateGraph
from langgraph.checkpoint.memory import MemorySaver
from langchain_experimental.agents import create_pandas_dataframe_agent
from langchain import hub
from typing_extensions import List, TypedDict
from pydantic import BaseModel, Field
from IPython.display import Image, display
from requests.exceptions import RequestException
import time
import logging
from google.colab import userdata # <-- Add this import
#HF_API_TOKEN="hf_EtVIdsMRwvCmgjeqoZimYzfRwDrfnk...."
#hf_token="hf_EtVIdsMRwvCmgjeqoZimYzfRwDrfnk...."
# Setup logging
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')

# 1. Set up the Hugging Face API token
#hf_token = os.environ.get('HF_API_TOKEN')
hf_token = userdata.get('HF_API_TOKEN')

if not hf_token:
    raise ValueError("Hugging Face API token not found. "
                     "Please set the HF_API_TOKEN environment variable.")

# Print first few characters to verify (don't print the whole token)
print(f"Loaded HF Token: {hf_token[:5]}...")

# 2. Load and split the document
try:
    data_url = "https://abc.xyz/assets/71/a5/78197a7540c987f13d247728a371/2024q3-alphabet-earnings-release.pdf"
    loader = PyPDFLoader(data_url)
    docs = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        add_start_index=True,
    )
    all_splits = text_splitter.split_documents(docs)
    print(f"Document split into {len(all_splits)} sub-documents.")
except Exception as e:
    logging.error(f"Error loading or splitting document: {e}")
    raise

# 3. Initialize embeddings with retry logic
from huggingface_hub import HfApi

def create_embeddings_with_retry(max_retries=3):
    # First, validate the token with a simple API call
    try:
        api = HfApi(token=hf_token)
        # Use model_info instead of get_model_status
        api.model_info("sentence-transformers/all-MiniLM-L6-v2")
        logging.info("HF Token validated successfully")
    except Exception as e:
        logging.error(f"Error validating HF token: {e}")
        raise ValueError("Invalid Hugging Face token or API access issue")
    # ...existing code...
    # Now set up embeddings with retry logic
    for attempt in range(max_retries):
        try:
            embeddings = HuggingFaceInferenceAPIEmbeddings(
                api_key=hf_token,
                model_name="sentence-transformers/all-MiniLM-L6-v2"
            )

            # Test the embeddings with a simple example
            test_embedding = embeddings.embed_query("Test query")
            if test_embedding and len(test_embedding) > 0:
                logging.info(f"Embeddings test successful, vector dimension: {len(test_embedding)}")
                return embeddings
            else:
                logging.warning("Embeddings test returned empty result")
        except RequestException as e:
            logging.warning(f"Attempt {attempt+1}/{max_retries} failed: {e}")
            if attempt < max_retries - 1:
                wait_time = 2 ** attempt  # Exponential backoff
                logging.info(f"Retrying in {wait_time} seconds...")
                time.sleep(wait_time)
            else:
                logging.error("All retry attempts failed")
                raise

    raise RuntimeError("Failed to initialize embeddings after multiple attempts")

# Initialize embeddings with retry
try:
    embeddings = create_embeddings_with_retry()
except Exception as e:
    logging.error(f"Failed to initialize embeddings: {e}")
    # Fallback to local embeddings if available
    try:
        logging.info("Trying fallback to local embeddings...")
        from langchain_community.embeddings import HuggingFaceEmbeddings

        embeddings = HuggingFaceEmbeddings(
            model_name="all-MiniLM-L6-v2"  # Will use local model if available
        )
        logging.info("Successfully initialized local embeddings")
    except Exception as local_e:
        logging.error(f"Local embeddings fallback also failed: {local_e}")
        raise

# 4. Create vector store
try:
    vector_store = Chroma.from_documents(
        documents=all_splits,
        embedding=embeddings
    )
    logging.info("Vector store created successfully")
except Exception as e:
    logging.error(f"Error creating vector store: {e}")
    raise

# 5. Define state and LLM
class State(TypedDict):
    category: bool
    question: str
    context: List[Document]
    answer: str

repo_id = "deepseek-ai/DeepSeek-R1-Distill-Qwen-32B"

try:
    llm = HuggingFaceEndpoint(
        repo_id=repo_id,
        temperature=0,
        huggingfacehub_api_token=hf_token,
        max_new_tokens=4000
    )
    llm = ChatHuggingFace(llm=llm)
    logging.info("LLM initialized successfully")
except Exception as e:
    logging.error(f"Error initializing LLM: {e}")
    raise

# 6. Define routing function
def route(state: State):
    output_parser = StrOutputParser()
    question = state["question"]

    template = f"""Here is a question from the user.
\nQuestion: {question}
\nYour job is to assign a category to this question
The question can be about RAG system and earnings report for Alphabet.
For rag category look for keywords like YouTube, Google, Alphabet, Revenue, etc.
For tabular category look for keywords like table, data, column, row, calendar, etc.
Assign the category 'rag' or 'tabular' based on the question.
The response must contain a single word containing the category: which can be rag or tabular
"""

    prompt = ChatPromptTemplate.from_template(template)
    llm_chain = prompt | llm | output_parser

    try:
        output = llm_chain.invoke({"question": question})
        response = output.strip().split("</think>")[-1].strip()
        logging.info(f"Route decision: {response}")
        return {"category": response}
    except Exception as e:
        logging.error(f"Error in routing: {e}")
        # Default to RAG if routing fails
        logging.info("Defaulting to 'rag' category due to error")
        return {"category": "rag"}

# 7. Select route function
def select_route(state: State):
    return state["category"]

# 8. Retrieve function
def retrieve(state: State):
    try:
        retrieved_docs = vector_store.similarity_search(state["question"])
        logging.info(f"Retrieved {len(retrieved_docs)} documents")
        return {"context": retrieved_docs}
    except Exception as e:
        logging.error(f"Error in retrieve function: {e}")
        # Return empty context if retrieval fails
        return {"context": []}

# 9. Pull prompt from hub
try:
    prompt = hub.pull("rlm/rag-prompt")
    example_messages = prompt.invoke(
        {"context": "(context goes here)", "question": "(question goes here)"}
    ).to_messages()
    assert len(example_messages) == 1
    logging.info("RAG prompt loaded successfully")
except Exception as e:
    logging.error(f"Error pulling prompt from hub: {e}")
    # Fallback to a simple prompt template
    template = """Answer the question based on the following context:

    Context: {context}

    Question: {question}

    Answer:"""
    prompt = ChatPromptTemplate.from_template(template)
    logging.info("Using fallback prompt template")

# 10. Generate function
def generate(state: State):
    try:
        docs_content = "\n\n".join(doc.page_content for doc in state["context"])
        messages = prompt.invoke({"question": state["question"], "context": docs_content})
        output = llm.invoke(messages).content
        response = output.strip().split("</think>")[-1].strip()
        logging.info("Generation completed successfully")
        return {"answer": response}
    except Exception as e:
        logging.error(f"Error in generate function: {e}")
        return {"answer": f"I encountered an error while trying to answer your question. Please try again or rephrase your question. Error details: {str(e)}"}

# 11. Tabular data handling (continued)
try:
    pathcsv = '/content/drive/MyDrive/Calendar/mayMonth.csv' #'/Users/alanforsythe/Library/Mobile Documents/com~apple~CloudDocs/abf_icloud3/AI_Pyth2025/DuelAgentPyth/mayMonth.csv'
    df = pd.read_csv(pathcsv)
    print(df.head())
    agent = create_pandas_dataframe_agent(llm, df,
                                        verbose=True,
                                        allow_dangerous_code=True)
    logging.info("Pandas DataFrame agent created successfully")
except Exception as e:
    logging.error(f"Error loading CSV or creating pandas agent: {e}")
    raise

# Define tabular response function that uses the agent
def tabular_response(state: State):
    try:
        logging.info(f"Processing tabular question: {state['question']}")
        response = agent.invoke(state['question'])
        logging.info("Agent response generated successfully")
        return {"answer": response["output"]}
    except Exception as e:
        logging.error(f"Error in tabular_response: {e}")
        return {"answer": f"I encountered an error while analyzing the data. Error details: {str(e)}"}

# 12. Build the graph
graph_builder = StateGraph(State)

# Add nodes
graph_builder.add_node("retrieve", retrieve)
graph_builder.add_node("generate", generate)
graph_builder.add_node("tabular", tabular_response)
graph_builder.add_node("router", route)

# Define edges
graph_builder.add_edge(START, "router")
graph_builder.add_conditional_edges(
    "router",
    select_route,
    {
        "rag": "retrieve",
        "tabular": "tabular"
    }
)
graph_builder.add_edge("retrieve", "generate")
graph_builder.add_edge("generate", END)
graph_builder.add_edge("tabular", END)

# Compile the graph
try:
    graph = graph_builder.compile()
    logging.info("Graph compiled successfully")
except Exception as e:
    logging.error(f"Error compiling graph: {e}")
    raise

# Save graph visualization to file
try:
    graph_image = graph.get_graph().draw_mermaid_png()
    with open('/Users/alanforsythe/Library/Mobile Documents/com~apple~CloudDocs/abf_icloud3/AI_Pyth2025/DuelAgentPyth/graph_visualization.png', 'wb') as f:
        f.write(graph_image)
    print("Graph visualization saved to 'graph_visualization.png'")
except Exception as e:
    logging.error(f"Error saving graph visualization: {e}")

# Interactive question answering loop
def interactive_qa():
    print("\n=== Interactive Question Answering System ===")
    print("Type 'exit' to quit")

    while True:
        try:
            user_question = input("\nEnter your question: ")

            if user_question.lower() in ['exit', 'quit', 'q']:
                print("Exiting...")
                break

            print("\nProcessing your question...")
            result = graph.invoke({"question": user_question})
            print(f"\nAnswer: {result['answer']}")

        except KeyboardInterrupt:
            print("\nExiting due to keyboard interrupt...")
            break
        except Exception as e:
            logging.error(f"Error processing question: {e}")
            print(f"\nError: {str(e)}")

# Run the interactive loop if this script is run directly
if __name__ == "__main__":
    # Optional: Run a test question first
    test_question = "Does this calendar have any events or activities that include the word 'bingo'?"
    print(f"\nRunning test question: '{test_question}'")
    try:
        result = graph.invoke({"question": test_question})
        print(f"Test answer: {result['answer']}")
    except Exception as e:
        logging.error(f"Error running test question: {e}")
        print(f"Test error: {str(e)}")

    # Start interactive mode
    interactive_qa()


/content/drive/MyDrive
fatal: destination path 'ICSA_DLcourse' already exists and is not an empty directory.
Loaded HF Token: hf_wm...
Document split into 33 sub-documents.


ERROR:root:All retry attempts failed
ERROR:root:Failed to initialize embeddings: Expecting value: line 1 column 1 (char 0)
/usr/local/lib/python3.11/dist-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


         Date DayOfWeek StartTime  EndTime                       EventName  \
0  2025-05-01  Thursday   9:00 AM      NaN           Morning Walk and Talk   
1  2025-05-01  Thursday   9:30 AM      NaN     Strength & Balance with Ann   
2  2025-05-01  Thursday  10:00 AM      NaN              Ambassador Meeting   
3  2025-05-01  Thursday  11:00 AM      NaN                       Ted Talks   
4  2025-05-01  Thursday   1:00 PM      NaN  Legends Concert: Glen Campbell   

  LocationCode   LocationName       Category Subcategory Notes  
0           CY      Courtyard       Wellness    Exercise   NaN  
1           AC  Activity Room       Wellness    Exercise   NaN  
2           CL    Club Lounge    Information     Meeting   NaN  
3           TH        Theatre  Entertainment     Lecture   NaN  
4           TH        Theatre  Entertainment       Music   NaN  


ERROR:root:Error saving graph visualization: [Errno 2] No such file or directory: '/Users/alanforsythe/Library/Mobile Documents/com~apple~CloudDocs/abf_icloud3/AI_Pyth2025/DuelAgentPyth/graph_visualization.png'



Running test question: 'Does this calendar have any events or activities that include the word 'bingo'?'


> Entering new AgentExecutor chain...
Okay, so I need to figure out if the calendar dataframe `df` has any events with the word 'bingo'. First, I should look at the columns. From the printout, the relevant column is `EventName`. 

I remember that in pandas, we can use string methods to check for substrings. So maybe using `str.contains('bingo')` on the `EventName` column would help. 

But wait, what if 'bingo' appears in other columns like Notes or Category? I should also check those. So I'll need to apply the check across multiple columns. 

I think the best approach is to create a boolean mask that checks each relevant column for 'bingo' and then see if any row has a True value. If any, then the answer is yes; otherwise, no.

So the steps are: 
1. Check each of the relevant columns (EventName, Notes, Category, Subcategory) for the substring 'bingo'.
2. Combine these checks with

ERROR:root:Error in tabular_response: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Parsing LLM output produced both a final answer and a parse-able action:: Alright, so I need to figure out how to show the lines in the calendar dataframe for May 27. Let's break this down step by step.

First, I know that the dataframe is called `df`, and it has a column named 'Date'. The dates are in the format 'YYYY-MM-DD', so May 27 would be '2025-05-27'. Wait, actually, in the sample data, the dates are '2025-05-01', so the year is 2025. So May 27 would be '2025-05-27'? Or is May represented as 05? Because in the sample, May 1 is '2025-05-01', so May 27 should be '2025-05-27'.

Next, I think I need to use pandas to filter the dataframe. The easiest way is to use boolean indexing. So the condition would be `df['Date'] == '2025-05-27'`. That should select all rows where 


Answer: I encountered an error while analyzing the data. Error details: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Parsing LLM output produced both a final answer and a parse-able action:: Alright, so I need to figure out how to show the lines in the calendar dataframe for May 27. Let's break this down step by step.

First, I know that the dataframe is called `df`, and it has a column named 'Date'. The dates are in the format 'YYYY-MM-DD', so May 27 would be '2025-05-27'. Wait, actually, in the sample data, the dates are '2025-05-01', so the year is 2025. So May 27 would be '2025-05-27'? Or is May represented as 05? Because in the sample, May 1 is '2025-05-01', so May 27 should be '2025-05-27'.

Next, I think I need to use pandas to filter the dataframe. The easiest way is to use boolean indexing. So the condition would be `df['Date'] == '2025-05-27'`.